In [1]:
import numpy as np
from keras.datasets import cifar10
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


ImportError: No module named protobuf

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()


In [ ]:
plt.imshow(X_train[0])

In [52]:
X_train_r = np.reshape(X_train, (X_train.shape[0], -1))
X_test_r = np.reshape(X_test, (X_test.shape[0], -1))
print (X_train_r.shape, X_test_r.shape)

(50000, 3072) (10000, 3072)


In [88]:
class KNearestNeighbor:
    def __init__(self):
        pass

    def train(self, X, y):
        """
        Train the classifier. For k-nearest neighbors this is just 
        memorizing the training data.
        Input:
        X - A num_train x dimension array where each row is a training point.
        y - A vector of length num_train, where y[i] is the label for X[i, :]
        """
        self.X_train = X
        self.y_train = y
    
    def predict(self, X, k=1):
        number_test = X.shape[0]
        number_train = self.X_train.shape[0]
        
        dists = np.zeros((number_test, number_train))
        
        for i in range(number_test):
            if i%10==0:
                print(i)
            for j in range(number_train):
                dists[i,j] = np.sum((X[i,:]-self.X_train[j,:])**2)
        
        num_test = dists.shape[0]
        y_pred = np.zeros(num_test)
        
        for i in range(num_test):
            closest_indices = np.argsort(dists[i,:])
            closest_indices = closest_indices[:k]
            closest_y = self.y_train[closest_indices]
            closest_y = closest_y.ravel()
            counts = np.bincount(closest_y)
            np.argmax(counts)
            y_pred[i] = np.argmax(counts)
          
#         for i in range(num_test):
#             closest_y = self.y_train[np.argsort(dists[i,:])[:k]]
#             u, indices = np.unique(closest_y, return_inverse=True)
#             print (closest_y)
#             assert 0
#             y_pred[i] = u[np.argmax(np.bincount(indices))]
            
        return y_pred


In [89]:
clf = KNearestNeighbor()
clf.train(X_train_r,y_train)

In [90]:
y_pred = clf.predict(X_test_r[:100],5)

0
10
20
30
40
50
60
70
80
90


In [91]:
np.sum(y_pred == y_test.ravel()[:100])

21

In [95]:
y_pred = clf.predict(X_test_r[:100],1)

0
10
20
30
40
50
60
70
80
90


In [96]:
np.sum(y_pred == y_test.ravel()[:100])

29

### Not good results, let's use some features

In [97]:
def color_histogram_hsv(im, nbin=15, xmin=0, xmax=255, normalized=True):
    """
    Compute color histogram for an image using hue.
    Inputs:
    - im: H x W x C array of pixel data for an RGB image.
    - nbin: Number of histogram bins. (default: 10)
    - xmin: Minimum pixel value (default: 0)
    - xmax: Maximum pixel value (default: 255)
    - normalized: Whether to normalize the histogram (default: True)
    Returns:
    1D vector of length nbin giving the color histogram over the hue of the
    input image.
    """
    ndim = im.ndim
    bins = np.linspace(xmin, xmax, nbin+1)
    hsv = matplotlib.colors.rgb_to_hsv(im/xmax) * xmax
    imhist, bin_edges = np.histogram(hsv[:,:,0], bins=bins, density=normalized)
    imhist = imhist * np.diff(bin_edges)

    # return histogram
    return imhist

In [98]:
X_train_f = []
for ind, im in enumerate(X_train):
    if ind % 10000 == 0:
        print (ind, X_train.shape[0])
    X_train_f.append(color_histogram_hsv(im))
    
    
X_test_f = []
for ind, im in enumerate(X_test):
    if ind % 10000 == 0:
        print (ind, X_test.shape[0])
    X_test_f.append(color_histogram_hsv(im))

0 50000
10000 50000
20000 50000
30000 50000
40000 50000
0 10000


In [99]:
X_train_f = np.array(X_train_f)
X_test_f = np.array(X_test_f)
print (X_train_f.shape, X_test_f.shape)

(50000, 15) (10000, 15)


In [100]:
clf = KNearestNeighbor()
clf.train(X_train_f,y_train)

In [101]:
y_pred = clf.predict(X_test_f[:100],1)

0
10
20
30
40
50
60
70
80
90


In [102]:
np.sum(y_pred == y_test.ravel()[:100])

23

## SKlearn 

In [103]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train_r,y_train)

/home/ubuntu/anaconda2/envs/deep-learning/lib/python3.5/site-packages/ipykernel/__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  app.launch_new_instance()


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [104]:
y_pred = neigh.predict(X_test_r[:100])

In [105]:
np.sum(y_pred == y_test.ravel()[:100])

31